In [8]:
import requests
import json
import pandas as pd

# loading test dataset (simulating production)
df_test = pd.read_csv('../data/test_raw.csv', index_col=0) 
#df_test = df_test.drop('response', axis=1).copy() #remove "response"
df_test = df_test.sample(10) #sample
df_test

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage
29850,410960,Male,54,1,41.0,1,1-2 Year,No,32965.0,122.0,188
79361,460471,Male,38,1,0.0,0,1-2 Year,Yes,30978.0,3.0,260
3842,384952,Male,70,1,33.0,1,1-2 Year,No,37980.0,124.0,289
62528,443638,Female,24,1,33.0,1,< 1 Year,No,44155.0,152.0,186
6497,387607,Male,25,1,28.0,0,< 1 Year,Yes,37064.0,152.0,29
86010,467120,Male,32,1,2.0,0,1-2 Year,No,26363.0,124.0,291
27332,408442,Male,24,1,13.0,0,< 1 Year,Yes,25748.0,152.0,175
17239,398349,Male,24,1,10.0,1,< 1 Year,No,38570.0,152.0,93
88696,469806,Male,23,1,8.0,1,< 1 Year,No,48579.0,152.0,228
25007,406117,Female,25,1,15.0,0,< 1 Year,Yes,38054.0,152.0,101


In [12]:
import pickle
import os
import pandas as pd
from flask import Flask, request, Response
#from healthinsurance.HealthInsurance import HealthInsurance

# load model in memory
model = pickle.load(open('models/xgbclassifier_model.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'models/xgbclassifier_model.pkl'

In [ ]:
# initialize API
app = Flask(__name__)

# create endpoint
@app.route('/predict', methods=['POST'])
def health_insurance_predict():
    test_json = request.get_json()

    if test_json: #there is data
        if isinstance(test_json, dict): #unique row 
            test_raw = pd.DataFrame(test_json, index = [0])
        else: # multiple rows
            test_raw = pd.DataFrame(test_json, columns = test_json[0].keys()) 
        
        #needed cause test_raw will be overwritten on pipeline
        test_raw_original = test_raw.copy()

        # instantiate HealthInsurance class
        pipeline = HealthInsurance()

        # data cleaning
        df1 = pipeline.data_cleaning(test_raw)
        #print('df1 done')
        #print(df1.head())
        #print(df1.head().values)
        
        # feature engineering
        df2 = pipeline.feature_engineering(df1)
        #print('df2 done')
        #print(df2.head())
        #print(df2.head().values)

        # data preparation
        df3 = pipeline.data_preparation(df2)
        #print('df3 done')
        #print(df3.head())
        #print(df3.head().values)
    
        # prediction
        df_response = pipeline.get_prediction(model, test_raw_original, df3)
        
        #returns a json
        return df_response
       
    else: #if empty:
        return Response('{}', status = 200, mimetype = 'application/json')

if __name__ == '__main__':
    port = os.environ.get( 'PORT', 5000 )
    app.run( host = '0.0.0.0', port=port )

In [5]:
# Builds json
data = json.dumps(df_test.to_dict(orient = 'records'))

In [6]:
#TEST LOCAL API

#API Call
url = 'http://192.168.1.21:5000/predict'  #local
header = {'Content-type': 'application/json'}

r = requests.post( url, data = data, headers = header )
print( 'Status Code {}'.format( r.status_code )) 

ConnectionError: HTTPConnectionPool(host='192.168.1.21', port=5000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C5AF9E24C0>: Failed to establish a new connection: [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu\r\ncorretamente após um período de tempo ou a conexão estabelecida falhou\r\nporque o host conectado não respondeu'))

In [ ]:
r.json()